In [ ]:
import galaxy, time, DM_Profiles, numpy, dill, os, scipy
from matplotlib import pyplot as plt

In [ ]:
bad_gs = [28, 35, 37, 83, 92, 93, 97]

In [ ]:
def dumper(gs, folder, minimum, maximum):
    """
    This function will write files with galaxies into a folder
    
    **Input**
    
    *gs* is a list of galaxies names
    
    *folder* name for a folder to dump to
    
    **Output**
    
    *unprocessed haloes list*
    
    """
    # first check the folder
    if not os.path.exists('./' + folder):
        os.mkdir('./' + folder)
    elif not os.path.isdir('./' + folder):
        print(folder + 'is not a folder')
        return None

    glist = []
    start_time = time.time()
    
    # unprocessed galaxies
    bad_gs = []
    # create a 2d array with indicies for files divided into groups of check
    ind = []
    total = len(gs)
    check = 10
    for k in range(total//check + (total%check != 0)):
        ind.append(range(total)[k * check: check * (k+1)])
    
    #create list of galaxies

    for l in ind:
        for galactica in l:

            try:
                galaxy_path = '/scratch/database/nihao/nihao_classic/' + gs[galactica] + '/' + gs[galactica] + '.01024'
                print(str(galactica + 1) + ' Creating ' + gs[galactica])
                gal = galaxy.galaxy(galaxy_path, minimum, maximum)
                glist.append(gal)
                print(str(round((galactica+1)/float(len(gs)) * 100)) + ' % done')
                print(time.time() - start_time)
                print('\n')

            except:
                print(galactica, " can't be done")
                bad_gs.append(galactica)
        
        #writing the info down

        with open('./' + folder + '/galaxies_' + str(ind.index(l)) + '.pkl', 'wb') as f:
            print('Wrinting a checkpoint')
            dill.dump(glist, f)
            glist = []
            print('checkpoint ' + str(ind.index(l) + 1) + ' created \n')
    return bad_gs

In [ ]:
def bline(pmodel, M200):
    
    a,b = .0, .0
    if pmodel in ('NFW', 'coreNFW', 'Lucky13', 'DC14', 'DC14_X-1.3' ):
        a = 0.830
        b = 0.098
        
    elif pmodel in ('Einasto', 'Einasto_ae'):
        a = 0.977
        b = 0.130
    
    M200 *= 0.6710431417754855/10**12
    C200 = 10**a/M200**b
    
    if pmodel in ('DC14','DC14_X-1.3'):
        M1 = 10**11.59; N = 0.0351; beta = 1.376; gamma = 0.608
        ratio = 2*N*((M200/M1)**(-beta) + (M200/M1)**gamma)**(-1)
        X = numpy.log10(ratio)
        C200 *= (1.0+0.00003*numpy.exp(3.4*(X+4.5)))
    return C200

In [ ]:
def final_graph(rows, columns, glist, to_save = False):

    #to make a 1d counter in 2d array
    counter = 0 

    #clearing out plt from  previous possible graphs and creating new figure
    plt.clf()
    fig, ax = plt.subplots(rows,columns, sharex = True, sharey = False, figsize = (15, 13))

    spearman = {}
    for a in range(rows):
        for b in range(columns):
            
            pmodel = DM_Profiles.models()[counter]
            C200s = []
            M200s = []
            for g in glist:
                M200 = g.variables[2]
                C200 = g.C200s[pmodel][0]
                M200s.append(M200)
                C200s.append(C200)
                ax[a][b].plot(M200, C200, 'b.')
                ax[a][b].errorbar(M200, C200, yerr = g.C200s[pmodel][1], ecolor = 'red')
            spearman.update({pmodel:(M200s, C200s)})
#                 ax[a][b].plot(g.variables[2], g.C200s[pmodel][0], 'b.')

            ax[a][b].grid(b=True, which='major', color='black', linestyle='-', linewidth = 0.5)
            ax[a][b].set_title(pmodel)
            ax[a][b].set_yscale('log')
            ax[a][b].set_xscale('log')
            if pmodel not in ('pISO', 'Burket'):
                ax[a][b].set_ylim(1, 100)
            else:
                ax[a][b].set_ylim(bottom = 1)
            
            #creating lines and corresponding error regions
            
            if pmodel not in ('pISO', 'Burket'):
                
                x = numpy.power(10, numpy.arange(9, 13, 0.1))
                y = bline(pmodel, numpy.power(10, numpy.arange(9, 13, 0.1)))
                e = 0.11#comes from the paper
                yme = y*10**e
                ype = y/10**e
                y2me = y*2*10**e
                y2pe = y/(2*10**e)
                ax[a][b].fill_between(x, yme, ype, alpha = 0.5, color = 'black')
                ax[a][b].fill_between(x, y2me, y2pe, alpha = 0.1, color = 'black')
                ax[a][b].plot(x, y)
        
            #creating labels only to the left
            if b == 0:
                ax[a][b].set_ylabel('log(C$_{200}$)')

            #when there are more panel slots than models just breaks
            if counter == len(DM_Profiles.models()) - 1:
                break
            counter += 1

    # creating labels only at the bottom
    for b in range(columns):
        ax[rows-1][b].set_xlabel('log(M$_{200}$/M$_{\odot}$)')

    fig.suptitle('final graph', fontsize = 16)
    fig.show()

#     saving the file
    if to_save:
        fig.savefig('./Graphs/final_plot.png')
        
    return spearman

In [ ]:
def reader(folder):
    """
    This function will read from pickle files from a given folder in the working directory
    
    **Input**
    
    *folder* as string with folder name
    
    
    **Output**
    
    *gals* list of processed galaxies
    
    """

    if not os.path.isdir('./' + folder):
        print(folder + ' is not a folder')
        return None
    glist = []
    
    for doc in os.listdir('./' + folder):
        with open('./' + folder + '/' + doc, 'rb') as f:
            a = dill.load(f)
            glist.append(a)

    gals = []
    for i in glist:
        for k in i:
            gals.append(k)
    for a in gals:
        for b in a.C200s:
            if a.C200s[b][0] > 10**5:
                gals.remove(a)
                
    return gals

In [ ]:
s = reader('0.01-0.2corrected')

In [ ]:
spearman = final_graph(3,3, s, to_save = 1)

In [ ]:
scor = {}
for pmodel in DM_Profiles.models():
    result = scipy.stats.spearmanr(spearman[pmodel][0], spearman[pmodel][1])
    scor.update({pmodel : result})

In [ ]:
scor

In [ ]:
scor